In [ ]:
import pickle
from collections import defaultdict

Load Data

In [ ]:
def load_artifacts():
    """Loads the processed data artifacts from disk."""
    print("--- Loading system artifacts... ---")
    try:
        with open("./artifacts/cranfield_vectors.pkl", 'rb') as f:
            doc_vectors = pickle.load(f)

        with open("./artifacts/cranfield_idf.pkl", 'rb') as f:
            idf_scores = pickle.load(f)
            
        print("Artifacts loaded successfully.")
        return doc_vectors, idf_scores
    except FileNotFoundError:
        print("Error: Could not find artifact files. Please run the '1_Index_Builder.ipynb' first.")
        return None, None

In [ ]:
load_artifacts()

Load queries

In [ ]:
def load_queries(filepath):
    """
    Parses the Cranfield queries file (.qry) to extract query IDs and their text.

    The function reads the file line by line, identifying new queries by the '.I' marker
    and accumulating the query text that follows the '.W' marker.
    """
    # Initialize an empty dictionary to store the final query data
    queries = {}
    
    # Use -1 as a placeholder to indicate that we haven't started reading the first query yet.
    current_id = -1
    
    # Initialize an empty string to accumulate the text for the query being processed.
    current_text = ""
    
    # Open the specified file for reading.
    with open(filepath, 'r') as f:
        # Iterate over each line in the file.
        for line in f:
            # Check if the line marks the beginning of a new query
            if line.startswith('.I'):
                # If current_id is not -1, it means we have just finished reading a
                # previous query, and its text needs to be saved before we start the new one.
                if current_id != -1:
                    # Save the accumulated text. .strip() removes whitespace from the ends,
                    # and .replace() handles text that spanned multiple lines.
                    queries[current_id] = current_text.strip().replace('\n', ' ')
                
                # Extract the new query ID from the line
                current_id = int(line.split()[1])
                
                # Reset the text accumulator to start fresh for this new query.
                current_text = ""
                
            # Check if the line is the '.W' marker, which indicates the start of the text.
            elif line.startswith('.W'):
                # This line is just a marker, so we don't need to do anything with it.
                pass
                
            # If the line is not a marker, it must be part of the query text.
            else:
                # Append the line to our accumulator for the current query.
                current_text += line
                
    # --- Final save after the loop ---
    # When the loop finishes, the text for the very last query is still in 'current_text'.
    # This final block ensures that the last query is also added to the dictionary.
    if current_id != -1:
        queries[current_id] = current_text.strip().replace('\n', ' ')
        
    # Return the completed dictionary of queries.
    return queries

In [13]:
queries =load_queries("./dataset/cran.qry")
    
# Print the first 3 queries to see if they look correct
for i in range(1, 5):
    if i in queries:
        print(f"Query ID {i}: {queries[i]}")

Query ID 1: what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .
Query ID 2: what are the structural and aeroelastic problems associated with flight of high speed aircraft .
Query ID 4: what problems of heat conduction in composite slabs have been solved so far .
